In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/opt/conda/envs/python2/bin/python'

In [2]:
import pyspark
from pyspark.sql import SQLContext, Row
from pyspark import SparkContext
sc = SparkContext('local[*]')

In [3]:
import plotly
from plotly.graph_objs import Scatter, Layout, Bar, Figure, Layout

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np

init_notebook_mode(connected=True)

In [4]:
filenames = [
            'orchestra_log_2_channel.csv', 
            'orchestra_log_3_channel.csv', 
            'orchestra_log_4_channel.csv', 
            'orchestra_log_5_channel.csv'
#            ]
            ,
# filenames = [
            'rlm_log_2_channel.csv', 
            'rlm_log_3_channel.csv', 
            'rlm_log_4_channel_diff.csv', 
            'rlm_log_5_channel.csv'
           ]
means2 = []
stdevs2 = []
means3 = []
stdevs3 = []

for filename in filenames:
    tf = sc.textFile(filename)
    rdd = tf.map(lambda x: (x.split(";")[0], x)).groupByKey().mapValues(list)
    hopping2 = rdd.filter(lambda x: len(x[1]) == 2)
    hopping3 = rdd.filter(lambda x: len(x[1]) == 3)

    latency2 = hopping2.map(lambda x: int(x[1][1].split(";")[1]) - int(x[1][0].split(";")[1])).collect()
    latency3 = hopping3.map(lambda x: int(x[1][2].split(";")[1]) - int(x[1][0].split(";")[1])).collect()
    
    means2.append(np.mean(latency2) / 1000000)
    means3.append(np.mean(latency3) / 1000000)
    stdevs2.append(np.std(latency2) / 1000000)
    stdevs3.append(np.std(latency3) / 1000000)
    
print 'means2  : ', means2
print 'stdevs2 : ', stdevs2
print 'means3  : ', means3
print 'stdevs3 : ', stdevs3

/opt/conda/envs/python2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning:

Mean of empty slice.

/opt/conda/envs/python2/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning:

Degrees of freedom <= 0 for slice



means2  :  [0.50424784507042253, 0.29142639130434783, 0.5212610491803279, 0.31962404166666669, 1.7543259324324325, 0.57806752459016397, 0.87213868518518511, 0.38884072916666668]
stdevs2 :  [0.96249663522418483, 0.31959351447577483, 0.8892092513314791, 0.40287274823597413, 4.1765136659689714, 0.42266272895434565, 1.3220068407370758, 0.32987094762729685]
means3  :  [1.1017507272727274, 0.62613615555555557, 0.88449544827586202, nan, 1.5898825000000001, nan, 1.1672411764705881, 0.74327255000000003]
stdevs3 :  [1.4085788025759958, 0.31217140581595415, 0.89418672768027463, nan, 3.3206666989623708, nan, 0.74813415533022876, 0.49552290627421808]


In [5]:
trace1 = Bar(
    x = filenames,
    y = means2,
    name='1 hopping',
    error_y=dict(
        type = 'data',
        array = stdevs2,
        visible = True
    )
)
trace2 = Bar(
    x = filenames,
    y = means3,
    name='2 hopping',
    error_y=dict(
        type = 'data',
        array = stdevs3,
        visible = True
    )
)

data = [trace1, trace2]
layout = Layout(
    barmode='group'
)
fig = Figure(data=data, layout=layout)
iplot(fig, filename='error-bar-bar')